In [ ]:
%load_ext autoreload
%autoreload 2

import hyp
import utils
import os
import json
import pandas as pd
import numpy as np

In [ ]:
settings = json.load(open('./settings.json'))

path = settings['general']['dataset_dir']
subs = ['05', '24', '37']
hours_counts = []

subs = utils.check_all_setting(subs, 'sub', path, prefix='sub')

for sub in subs:
    settings['general']['sub'] = sub
    settings['general']['ses'] = '40w'
    settings['general']['run'] = 'all'

    os.remove('./settings.json')
    with open('./settings.json', 'w') as f:
        json.dump(settings, f)

    sub = settings['general']['sub']
    ses = settings['general']['ses']

    path = os.path.join(settings['general']['dataset_dir'], settings['general']['eeg_subdir'].split('ses')[0].replace(
            '<SUB>', sub))

    ses = utils.check_all_setting(ses, 'ses', path, suffix='w')

    for _ses in ses:
        run = settings['general']['run']

        output_dir = utils.create_output_dir(settings['general']['output_dir'], sub, _ses)

        path = os.path.join(settings['general']['dataset_dir'], settings['general']['eeg_subdir'].replace(
            '<SUB>', sub).replace('<SES>', _ses))
        
        run = utils.check_all_setting(run, 'run', path, suffix=settings['hyp']['suffix'])

        dfs = []

        for _run in run:
            filename = utils.get_filename(settings, _ses, _run, settings['hyp']['suffix'])
            filename = os.path.join(path, filename)

            dfs.append(hyp.get_hyp_df(filename, settings))

        df = pd.concat(dfs, ignore_index=True)
        df = df[['description', 'date']]

        df = hyp.normalize_dataframe(df, settings)

        hours_counts.append(hyp.count_full_hours(df))

hours_counts = np.sum(hours_counts, axis=0)
hours_counts = [int(count) for count in hours_counts]
print(hours_counts)

In [ ]:
import matplotlib.pyplot as plt

hours = range(0, 24)

plt.xlabel('Hours')
plt.ylabel('Counts')
plt.xticks(hours)
plt.yticks(range(0, np.max(hours_counts) + 1))
plt.title('Recording Hours (tol = 30m)')
plt.bar(hours, hours_counts)